##  Import dependencies

In [13]:
import requests
import ipywidgets as widgets
from IPython.display import display

In [14]:
from bqplot import pyplot as plt
import bqplot
import numpy as np

## setup connection to the server

In [15]:
local_url = "http://127.0.0.1:5000/los-cangri/"
online_url = "https://los-cangris-d8db459397d0.herokuapp.com/los-cangri/"
local = False

base_url = local_url if local else online_url 

#helper url's
warehouse_url = base_url+'warehouse'
most_url = base_url+'most/'
least_url = base_url+'least/'

In [16]:
data = requests.get(warehouse_url)
data

<Response [200]>

## helper functions

### global statistics
a. /most/rack – Top 10 warehouses with the most racks

b. /most/incoming – Top 5 warehouses with the most incoming transactions

c. /most/deliver – Top 5 warehouses that delivers the most exchanges

d. /most/transactions – Top 3 users that made the most transactionse.

e. /least/outgoing – Top 3 warehouses with the least outgoing transactions

f. /most/city – Top 3 warehouses’ cities with the most transactions


In [17]:
def get_most_racks():
    data = requests.get(most_url+'rack')
    return data.json()

def get_most_incoming():
    data = requests.get(most_url+'incoming')
    return data.json()

def get_most_deliver():
    data = requests.get(most_url+'deliver')
    return data.json()  

def get_most_transactions():
    data = requests.get(most_url+'deliver')
    return data.json()  

def get_least_outgoing():
    data = requests.get(least_url+'deliver')
    return data.json()  

def get_most_city():
    data = requests.get(most_url+'city')
    return data.json()  



In [18]:
def display_warehouse_with_most_racks():
    racks = get_most_racks()['Warehouses']
    wids = [pair['wid'] for pair in racks]
    counts = [pair['count'] for pair in racks]
    fig = plt.figure()
    bar_chart = plt.bar(
        wids, 
        counts, 
        scales={'x':bqplot.OrdinalScale()}
    )
    plt.xlabel('WID')
    plt.ylabel('Count')
    plt.title('Warehouses with most racks')

    plt.show()

display_warehouse_with_most_racks()

In [19]:
def display_warehouse_with_most_incoming_transactions():
    racks = get_most_incoming()['Warehouses']
    wids = [pair['wid'] for pair in racks]
    counts = [pair['count'] for pair in racks]
    fig = plt.figure()
    bar_chart = plt.bar(
        wids, 
        counts, 
        scales={'x':bqplot.OrdinalScale()}
    )
    plt.xlabel('WID')
    plt.ylabel('Count')
    plt.title('Warehouses with most incoming transactions')

    plt.show()

display_warehouse_with_most_incoming_transactions()

### local statistics

a. /warehouse/<id>/profit – Warehouse’s year profit

b. /warehouse/<id>/rack/lowstock – Top 5 racks with quantity under the 25% capacity 
threshold

c. /warehouse/<id>/rack/material – Bottom 3 part’s type/material in the warehouse

d. /warehouse/<id>/rack/expensive – Top 5 most expensive racks in the warehouse

e. /warehouse/<id>/transaction/suppliers – Top 3 supplier that supplied to the warehouse 

f. /warehouse/<id>/transaction/leastcost – Top 3 days with the smallest incoming 
transactions’ cost

g. /warehouse/<id>/users/receivesmost – Top 3 users that receives the most exchanges

In [20]:
def get_warehouse_profit(wid:int, uid:int):
    perms = {'User_id': uid}
    data = requests.post(warehouse_url+f'/{wid}/profit', json=perms)
    return data.json()

def get_warehouse_data(wid:int):
    data = requests.get(warehouse_url+f'/{wid}')
    return data.json()

In [21]:
widgets.interact(get_warehouse_data, wid=1)
# widgets.interact(get_warehouse_profit, wid=1, uid=1)

interactive(children=(IntSlider(value=1, description='wid', max=3, min=-1), Output()), _dom_classes=('widget-i…

<function __main__.get_warehouse_data(wid: int)>

### dropdown code

In [22]:
local_choices = ['profit', 'lowstock', 'material', 
                 'expensive', 'suppliers', 'leastcost'
                 , 'receivesmost']
global_choices = ['mostrack', 'mostincoming', 'mostdeliver',
                  'mosttransactions', 'leastoutgoing', 'mostcity']

part_info_choices = ['Part Prices', 'Parts Supplied by Supplier', 'Parts in a Warehouse']

main_choices = ['Local Statistics', 'Global Statistics', 'Transactions', 'Part Information']


main_selection = widgets.Dropdown(options = main_choices, value=main_choices[0])
entity_selection = widgets.Dropdown(options =  ['Warehouse','User'])
statistic_choice = widgets.Dropdown(
    options = local_choices)
choice_output = widgets.Output()

#Supplier selection for parts and Wid selection for parts and transactions
supplier_choice = widgets.BoundedIntText(value=1, description='Supplier:', min=1)
wid_choice = widgets.BoundedIntText(value=1, description='Wid:', min=1)

#display(main_selection, statistic_choice)
def reset_choices():
    supplier_choice.value = 1
    wid_choice.value = 1
    supplier_choice.layout.display = 'none'
    wid_choice.layout.display = 'none'
    statistic_choice.layout.display = None

def display_part_selections(change):
    reset_choices()
    current = change.new
    if current == 'Parts Supplied by Supplier':
        supplier_choice.layout.display = None
    if current == 'Parts in a Warehouse':
        wid_choice.layout.display = None

def update_statistic_choice(change):
    reset_choices()
    current = change.new
    new_choices = None
    if current == 'Local Statistics':
        new_choices = local_choices
    elif current == 'Global Statistics':
        new_choices = global_choices
    elif current == 'Part Information':
        new_choices = part_info_choices
    elif current == 'Transactions':
        statistic_choice.layout.display = 'none'
        wid_choice.layout.display = None
        return
    statistic_choice.options = new_choices
    statistic_choice.value = new_choices[0]
reset_choices()
display_box = widgets.VBox([main_selection, statistic_choice, supplier_choice, wid_choice])
display(display_box)
#display(main_selection, statistic_choice, supplier_choice, wid_choice)
#sets main_selection callback to update the choice of statistics
main_selection.observe(update_statistic_choice, names='value')
statistic_choice.observe(display_part_selections, names='value')



    

#### dropdown graph selecion

In [23]:
graph_output = widgets.Output()

def view_statistic(option):
    graph_output.clear_output()
    curr = option.new
    with graph_output:
        if curr == 'mostrack':
            display_warehouse_with_most_racks()
        elif curr == 'mostincoming':
            display_warehouse_with_most_incoming_transactions()
    
    
    pass    
statistic_choice.observe(view_statistic, names = 'value')
display(graph_output)

Output()